# One to Many Relationship In DynamoDB - Method 1, Multiple Tables

This notebook shows the first method to define a data model for one-to-many relationship in DynamoDB.

Reference:

- [How to model one-to-many relationships in DynamoDB](https://www.alexdebrie.com/posts/dynamodb-one-to-many/#denormalization-by-using-a-complex-attribute)
- [Best practices for modeling relational data in DynamoDB](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/bp-relational-modeling.html)


# Import Python Libraries

In [1]:
import typing as T
import enum
from datetime import datetime

import dataclasses
import pynamodb_mate as pm
import pynamodb.exceptions as exc
from moto import mock_dynamodb

from rich import print as rprint

- One employee belongs to only one department.
- One department may have many employees.

# Configure AWS Connection

In [2]:
# create a DynamoDB connection, ensure that your default AWS credential is right
# if you are using mock, then this line always works
connect = pm.Connection()

In [3]:
# use moto to mock DynamoDB, it is an in-memory implementation of DynamoDB
# you can also use the real DynamoDB table by just comment out the below two line
mock = mock_dynamodb()
mock.start()

# Type Hint

In [4]:
REQUIRED_STR = T.Union[str, pm.UnicodeAttribute]
OPTIONAL_STR = T.Optional[REQUIRED_STR]
REQUIRED_INT = T.Union[int, pm.NumberAttribute]
OPTIONAL_INT = T.Optional[REQUIRED_INT]
REQUIRED_DATETIME = T.Union[datetime, pm.UTCDateTimeAttribute]
OPTIONAL_DATETIME = T.Optional[REQUIRED_DATETIME]

# Declare Department

In [19]:
class Department(pm.Model):
    class Meta:
        table_name = "department"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    department_id: REQUIRED_STR = pm.UnicodeAttribute(hash_key=True)
    department_name: REQUIRED_STR = pm.UnicodeAttribute()

Department.create_table(wait=True)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


# Declare Employee

In [20]:
class DepartmentEmployeeIndex(pm.GlobalSecondaryIndex):
    class Meta:
        index = "department-employee-index"
        projection = pm.AllProjection

    department_id: REQUIRED_STR = pm.UnicodeAttribute(hash_key=True, null=True)


class Employee(pm.Model):
    class Meta:
        table_name = "employee"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    employee_id: REQUIRED_STR = pm.UnicodeAttribute(hash_key=True)
    employee_name: REQUIRED_STR = pm.UnicodeAttribute()
    department_id: OPTIONAL_STR = pm.UnicodeAttribute(null=True)
    
    department_index = DepartmentEmployeeIndex()

Employee.create_table(wait=True)

# Business Operation

In [21]:
class OP:
    def new_department(
        self,
        department_id: str, 
        department_name: str,
    ) -> T.Optional[Department]:
        department = Department(
            department_id=department_id,
            department_name=department_name,
        )
        try:
            res = department.save(
                condition= (~Department.department_id.exists()),
            )
            return department
        except exc.PutError as e:
            return None

    def new_employee(
        cls, 
        employee_id: str, 
        employee_name: str, 
        department_id: T.Optional[str] = None,
    ) -> T.Optional[Employee]:
        employee = Employee(
            employee_id=employee_id,
            employee_name=employee_name,
            department_id=department_id,
        )
        try:
            res = employee.save(
                condition= (~Employee.employee_id.exists()),
            )
            return employee
        except exc.PutError as e:
            return None
            
    def all_department(self) -> list[Department]:
        return list(Department.scan())

    def all_employee(self) -> list[Employee]:
        return list(Employee.scan())
    
    def find_all_employee_in_given_department(
        self,
        department_id: str,
    ) -> list[Employee]:
        return list(DepartmentEmployeeIndex.query(hash_key="d-1"))

    def change_department(self, employee_id: str, department_id: str) -> bool:
        """
        Return a boolean value to indicate whether the update is successful.
        """
        try:
            Employee(employee_id=employee_id).update(
                actions=[
                    Employee.department_id.set(department_id),
                ],
                condition=(Employee.employee_id.exists()),
            )
            return True
        except exc.UpdateError as e:
            return False

op = OP()
    

# Create Dummy Data

In [23]:
department = op.new_department(department_id="d-1", department_name="HR")
department = op.new_department(department_id="d-2", department_name="IT")

employee = op.new_employee(employee_id="e-1", employee_name="Alice", department_id="d-1")
employee = op.new_employee(employee_id="e-2", employee_name="Bob", department_id="d-1")

employee = op.new_employee(employee_id="e-3", employee_name="Cathy", department_id="d-2")
employee = op.new_employee(employee_id="e-4", employee_name="David", department_id="d-2")
employee = op.new_employee(employee_id="e-5", employee_name="Edward", department_id="d-2")

# Show all Data

In [24]:
rprint("------ Department ------")
for department in op.all_department():
    rprint(department.to_dict())
rprint("------ Employee ------")
for employee in op.all_employee():
    rprint(employee.to_dict())

------ Department ------

{'department_id': 'd-1', 'department_name': 'HR'}

{'department_id': 'd-2', 'department_name': 'IT'}

------ Employee ------

{'department_id': 'd-2', 'employee_id': 'e-1', 'employee_name': 'Alice'}

{'department_id': 'd-1', 'employee_id': 'e-2', 'employee_name': 'Bob'}

{'department_id': 'd-2', 'employee_id': 'e-3', 'employee_name': 'Cathy'}

{'department_id': 'd-2', 'employee_id': 'e-4', 'employee_name': 'David'}

{'department_id': 'd-2', 'employee_id': 'e-5', 'employee_name': 'Edward'}

# Find all Employee in the Given Department

In [25]:
rprint("--- Employee in HR department ---")
for employee in op.find_all_employee_in_given_department(department_id="d-1"):
    print(type(employee))
    rprint(employee.to_dict())
    
rprint("--- Employee in IT department ---")
for employee in DepartmentEmployeeIndex.query(hash_key="d-2"):
    rprint(employee.to_dict())

--- Employee in HR department ---

<class '__main__.Employee'>


{'department_id': 'd-1', 'employee_id': 'e-2', 'employee_name': 'Bob'}

--- Employee in IT department ---

{'department_id': 'd-2', 'employee_id': 'e-1', 'employee_name': 'Alice'}

{'department_id': 'd-2', 'employee_id': 'e-3', 'employee_name': 'Cathy'}

{'department_id': 'd-2', 'employee_id': 'e-4', 'employee_name': 'David'}

{'department_id': 'd-2', 'employee_id': 'e-5', 'employee_name': 'Edward'}

# Change Department

In [26]:
flag = op.change_department(employee_id="e-1", department_id="d-2")
if flag:
    rprint("Succeeded")
else:
    rprint("Failed")

Succeeded

In [27]:
flag = op.change_department(employee_id="e-999", department_id="d-2")
if flag:
    rprint("Succeeded")
else:
    rprint("Failed")

Failed

In [28]:
print("--- New department for Alice should be d-2 ---")
rprint(Employee.get("e-1").to_dict())

--- New department for Alice should be d-2 ---


{'department_id': 'd-2', 'employee_id': 'e-1', 'employee_name': 'Alice'}